# 测试流程示例

演示如何为 Xenium 与 Visium 数据计算基因程序分数、生成真值宽表，并导出解卷积输入。


## 目录
- 环境配置与基因集准备
- Xenium 基因程序打分
- Xenium 真值生成
- Visium 基因程序打分
- 数据对齐与 NPZ 导出


## 1. 环境配置与基因集设定

导入所需依赖，注册包路径，并定义用于测试的 4 组基因程序。


In [1]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

import sys
import os
import numpy as np
import pandas as pd
import scanpy as sc

# -------------------------------------------------------------------------
# 1. 添加模块路径并导入函数
# -------------------------------------------------------------------------
sys.path.append("/home/vs_theg/ST_program/CellType_GP/CellType-GP/")
from score_gene_program import score_gene_programs
from compute_truth_score import compute_truth_score

# -------------------------------------------------------------------------
# 2. 定义所有 17 个 Cluster 的基因集
# -------------------------------------------------------------------------
DCIS_1_genes = [
    'HPX', 'CEACAM6', 'ESR1', 'HOOK2', 'CEACAM8', 'GATA3', 'TFAP2A', 'FLNB', 'KLF5', 'CD9',
    'TPD52', 'CLDN4', 'SMS', 'DNTTIP1', 'QARS', 'C6orf132', 'KLRF1', 'LYPD3', 'SDC4', 'RHOH'
]

DCIS_2_genes = [
    'AGR3', 'S100A14', 'CEACAM8', 'KRT8', 'CCND1', 'CDH1', 'TCIM', 'AQP3', 'TACSTD2', 'LYPD3',
    'SERHL2', 'ESR1', 'CEACAM6', 'BACE2', 'DSP', 'SERPINA3', 'RORC', 'ERBB2', 'CLDN4', 'DMKN'
]

Prolif_Invasive_genes = [
    'CENPF', 'MKI67', 'TOP2A', 'PCLAF', 'STC1', 'RTKN2', 'TUBA4A', 'MDM2', 'HMGA1', 'C2orf42',
    'POLR2J3', 'PTRHD1', 'SRPK1', 'EIF4EBP1', 'SQLE', 'SH3YL1', 'THAP2', 'NPM3', 'LAG3', 'FOXA1'
]

Invasive_Tumor_genes = [
    'ABCC11', 'SERHL2', 'TCIM', 'FASN', 'AR', 'PTRHD1', 'TRAF4', 'USP53', 'SCD', 'SQLE',
    'MYO5B', 'DNAAF1', 'FOXA1', 'EPCAM', 'CTTN', 'MLPH', 'ELF3', 'ANKRD30A', 'ENAH', 'KARS'
]

# 合并所有基因集用于评分
gene_sets_to_score = {
    'DCIS_1_score': DCIS_1_genes,
    'DCIS_2_score': DCIS_2_genes,
    'Prolif_Invasive_score': Prolif_Invasive_genes,
    'Invasive_Tumor_score': Invasive_Tumor_genes,
}


/home/vs_theg/anaconda3/envs/DL/lib/python3.9/site-packages/xarray_schema/__init__.py:1: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import DistributionNotFound, get_distribution
/home/vs_theg/anaconda3/envs/DL/lib/python3.9/site-packages/numba/core/decorators.py:246: RuntimeWarning: nopython is set for njit and is ignored
  warnings.warn('nopython is set for njit and is ignored', RuntimeWarning)


## 2. Xenium 基因程序打分

在 Xenium 单细胞矩阵上运行 `score_gene_programs`，并把结果落盘。


In [2]:
os.chdir('/home/vs_theg/ST_program/CellType_GP/DATA/')
adata_x = sc.read("/home/vs_theg/ST_program/CellType_GP/DATA/xdata.h5")
score_gene_programs(adata_x, gene_sets_to_score, platform="xenium", output_dir="xenium_scores")
adata_x.write("/home/vs_theg/ST_program/CellType_GP/DATA/xdata_processed.h5")

🔹 正在处理: DCIS_1_score
✅ 完成: DCIS_1_score (20 基因)

🔹 正在处理: DCIS_2_score
✅ 完成: DCIS_2_score (20 基因)

🔹 正在处理: Prolif_Invasive_score
✅ 完成: Prolif_Invasive_score (20 基因)

🔹 正在处理: Invasive_Tumor_score
✅ 完成: Invasive_Tumor_score (20 基因)


🎉 所有基因集打分完成（使用平均表达），结果已保存至 xenium_scores/



## 3. 生成 Xenium 真值宽表

合并 Xenium 注释后计算聚合得分，输出到 `truth_output/` 目录。


In [3]:

xenium_to_visium_transcript_mapping = pd.read_csv(
    '/home/vs_theg/ST_program/CellType_GP/DATA/xenium_to_visium_transcript_mapping.csv'
)
possible_keys = ["cell_id", "Barcode", "cell_ID", "xenium_cell_id"]
left_key = next((k for k in possible_keys if k in adata_x.obs.columns), None)
if left_key is None:
    raise ValueError("❌ 在 adata_x.obs 中找不到匹配的 cell id 列。")

adata_x.obs = adata_x.obs.merge(
    xenium_to_visium_transcript_mapping[["xenium_cell_id", "transcript_level_visium_barcode"]],
    how="left",
    left_on=left_key,
    right_on="xenium_cell_id"
)
if "xenium_cell_id" in adata_x.obs.columns:
    adata_x.obs.drop(columns=["xenium_cell_id"], inplace=True)

before = adata_x.n_obs
adata_x = adata_x[adata_x.obs["transcript_level_visium_barcode"].notna()].copy()
after = adata_x.n_obs
print(f"✅ 去除无 transcript_level_visium_barcode 的细胞：删除 {before - after}，剩余 {after}")

compute_truth_score(adata_x)

/home/vs_theg/anaconda3/envs/DL/lib/python3.9/site-packages/anndata/_core/aligned_df.py:68: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


✅ 去除无 transcript_level_visium_barcode 的细胞：删除 22938，剩余 135092
🚀 开始计算 Xenium truth score, 输出路径: ./truth_output

🧾 当前 adata.obs 列如下（可直接复制粘贴）：

cell_id, transcript_counts, control_probe_counts, control_codeword_counts, total_counts, cell_area, nucleus_area, region, Barcode, Cluster, broad_annotation, Stromal_score, Prolif_Invasive_Tumor_score, Perivascular_Like_score, Myoepi_KRT15_score, Myoepi_ACTA2_score, Mast_Cells_score, Macrophages_2_score, Macrophages_1_score, LAMP3_DCs_score, IRF7_DCs_score, Invasive_Tumor_score, Endothelial_score, DCIS_2_score, DCIS_1_score, CD8_T_Cells_score, CD4_T_Cells_score, B_Cells_score, DCIS_1_score_norm, DCIS_2_score_norm, Prolif_Invasive_score_norm, Invasive_Tumor_score_norm, transcript_level_visium_barcode

💡 提示：你可以复制上面这一行，然后粘贴要删除的列（或部分列）

✅ 未删除任何列
✨ 清洗完成：保留 33 列。

🧩 检测到 4 个得分列：['DCIS_1_score_norm', 'DCIS_2_score_norm', 'Prolif_Invasive_score_norm', 'Invasive_Tumor_score_norm'] ...
✅ 分组平均完成: 27671 行


/home/vs_theg/ST_program/CellType_GP/CellType-GP/compute_truth_score.py:65: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df.groupby(['transcript_level_visium_barcode', 'broad_annotation'])[score_cols]
/home/vs_theg/ST_program/CellType_GP/CellType-GP/compute_truth_score.py:71: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df.groupby(['transcript_level_visium_barcode', 'broad_annotation'])
/home/vs_theg/ST_program/CellType_GP/CellType-GP/compute_truth_score.py:89: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=Fa

✅ 宽表完成: 3953 × 29
💾 保存结果：
  ├─ 细胞级 truth_score：./truth_output/truth_score.csv
  ├─ 分组均值 truth_result：./truth_output/truth_result_grouped.csv
  └─ 宽表 truth_result(wide)：./truth_output/truth_result(wide).csv

🎉 Xenium truth score 计算完成！


,spot,B_Cells+DCIS_1_score_norm,DCIS+DCIS_1_score_norm,Endothelial+DCIS_1_score_norm,Invasive_Tumor+DCIS_1_score_norm,Myeloid+DCIS_1_score_norm,Stromal+DCIS_1_score_norm,T_cells+DCIS_1_score_norm,B_Cells+DCIS_2_score_norm,DCIS+DCIS_2_score_norm,...,Myeloid+Invasive_Tumor_score_norm,Stromal+Invasive_Tumor_score_norm,T_cells+Invasive_Tumor_score_norm,B_Cells+Prolif_Invasive_score_norm,DCIS+Prolif_Invasive_score_norm,Endothelial+Prolif_Invasive_score_norm,Invasive_Tumor+Prolif_Invasive_score_norm,Myeloid+Prolif_Invasive_score_norm,Stromal+Prolif_Invasive_score_norm,T_cells+Prolif_Invasive_score_norm
0,AACACGTGCATCGCAC-1,0.375284,NaN,NaN,NaN,0.000000,0.220213,NaN,0.147681,NaN,...,0.056702,0.220103,NaN,0.404542,NaN,NaN,NaN,0.000000,0.217262,NaN
1,AACACTTGGCAAGGAA-1,NaN,0.575892,0.277421,NaN,0.108926,0.146012,0.116139,NaN,0.845507,...,0.094999,0.161601,0.097661,NaN,0.310731,0.177289,NaN,0.184925,0.196339,0.217015
2,AACAGGAAGAGCATAG-1,NaN,NaN,NaN,NaN,NaN,0.321083,NaN,NaN,NaN,...,NaN,0.344991,NaN,NaN,NaN,NaN,NaN,NaN,0.231498,NaN
3,AACAGGATTCATAGTT-1,NaN,NaN,0.247145,NaN,0.062199,0.276357,0.171217,NaN,NaN,...,0.151933,0.390091,0.205113,NaN,NaN,0.275294,NaN,0.212327,0.322324,0.331868
4,AACAGGTTATTGCACC-1,NaN,NaN,NaN,NaN,0.076516,0.134703,NaN,NaN,NaN,...,0.162328,0.183649,NaN,NaN,NaN,NaN,NaN,0.154542,0.201818,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3948,TGTTGGAACCTTCCGC-1,NaN,NaN,NaN,NaN,NaN,0.197189,NaN,NaN,NaN,...,NaN,0.303076,NaN,NaN,NaN,NaN,NaN,NaN,0.321613,NaN
3949,TGTTGGAACGAGGTCA-1,NaN,NaN,NaN,NaN,0.128381,0.203931,NaN,NaN,NaN,...,0.104531,0.169283,NaN,NaN,NaN,NaN,NaN,0.141015,0.187936,NaN
3950,TGTTGGAAGCTCGGTA-1,0.160356,0.556254,0.238386,NaN,0.105313,0.131702,0.156128,0.104129,0.735014,...,0.089908,0.151088,0.067906,0.180778,0.331768,0.193754,NaN,0.212019,0.204104,0.212550
3951,TGTTGGATGGACTTCT-1,NaN,NaN,0.157251,0.473101,0.089738,0.169298,0.290752,NaN,NaN,...,0.137208,0.295425,0.324691,NaN,NaN,0.114060,0.524094,0.216858,0.213031,0.333305


## 4. Visium 基因程序打分

对 Visium 空间数据重复基因程序打分，并保存处理后的 AnnData 文件。


In [4]:
adata_v = sc.read("/home/vs_theg/ST_program/CellType_GP/DATA/vdata.h5")
score_gene_programs(adata_v, gene_sets_to_score, platform="visium", output_dir="visium_scores")
adata_v.write("/home/vs_theg/ST_program/CellType_GP/DATA/vdata_processed.h5")


🔹 正在处理: DCIS_1_score
✅ 完成: DCIS_1_score (20 基因)

🔹 正在处理: DCIS_2_score
✅ 完成: DCIS_2_score (20 基因)

🔹 正在处理: Prolif_Invasive_score
⚠️ 缺失基因 (Prolif_Invasive_score): POLR2J3
✅ 完成: Prolif_Invasive_score (19 基因)

🔹 正在处理: Invasive_Tumor_score
✅ 完成: Invasive_Tumor_score (20 基因)


🎉 所有基因集打分完成（使用平均表达），结果已保存至 visium_scores/



## 5. 数据对齐与 NPZ 导出

同步 Visium 得分、空间坐标与细胞类型比例矩阵，最终生成 `spot_data_full.npz`。


In [5]:
spot_cluster_fraction_matrix = pd.read_csv(
    '/home/vs_theg/ST_program/CellType_GP/DATA/spot_cluster_fraction_matrix.csv', index_col=0
)

target_spots = spot_cluster_fraction_matrix.index
adata_v = adata_v[target_spots.intersection(adata_v.obs_names)].copy()
adata_v = adata_v[target_spots]  # 保证顺序一致
print(f"✅ Visium 数据已对齐，共 {adata_v.n_obs} 个 spot。")

score_cols = [c for c in adata_v.obs.columns if c.endswith("_score_norm")]
visium_score = adata_v.obs[score_cols].values.T
coords = adata_v.obsm["spatial"]
spot_names = spot_cluster_fraction_matrix.index.values
celltype_names = np.array(['B_Cells', 'DCIS', 'Endothelial', 'Invasive_Tumor',
                           'Myeloid', 'Stromal', 'T_cells'])
program_names = np.array(score_cols)
print(program_names)

import pandas as pd
coords_df = pd.DataFrame(coords, columns=["x", "y"])
coords_df.to_csv("/home/vs_theg/ST_program/CellType_GP/DATA/vdata_spatial_coords.csv", index=False)
print("✅ 已保存到 vdata_spatial_coords.csv，形状：", coords_df.shape)

np.savez_compressed(
    '/home/vs_theg/ST_program/CellType_GP/DATA/spot_data_full.npz',
    spot_cluster_fraction_matrix=spot_cluster_fraction_matrix.values,
    coords=coords,
    visium_score=visium_score,
    spot_names=spot_names,
    celltype_names=celltype_names,
    program_names=program_names
)
print("🎉 成功保存：/home/vs_theg/ST_program/CellType_GP/DATA/spot_data_full.npz")

print("✅ 矩阵形状：")
print("spot_cluster_fraction_matrix:", spot_cluster_fraction_matrix.shape)
print("visium_score:", visium_score.shape)
print("coords:", coords.shape)


✅ Visium 数据已对齐，共 3953 个 spot。
['DCIS_1_score_norm' 'DCIS_2_score_norm' 'Prolif_Invasive_score_norm'
 'Invasive_Tumor_score_norm']
✅ 已保存到 vdata_spatial_coords.csv，形状： (3953, 2)
🎉 成功保存：/home/vs_theg/ST_program/CellType_GP/DATA/spot_data_full.npz
✅ 矩阵形状：
spot_cluster_fraction_matrix: (3953, 7)
visium_score: (4, 3953)
coords: (3953, 2)


# 6. 输出文件的地址

- `/home/vs_theg/ST_program/CellType_GP/DATA/xenium_scores/`
- `/home/vs_theg/ST_program/CellType_GP/DATA/xdata_processed.h5`

## 真值文件
- `/home/vs_theg/ST_program/CellType_GP/DATA/truth_output/` 
包含：
  - `truth_result(wide).csv`
  - `truth_score.csv`
  - `truth_result_grouped.csv`  

- `/home/vs_theg/ST_program/CellType_GP/DATA/visium_scores/`
- `/home/vs_theg/ST_program/CellType_GP/DATA/vdata_processed.h5`

## 训练输入文件
- `/home/vs_theg/ST_program/CellType_GP/DATA/vdata_spatial_coords.csv`
- `/home/vs_theg/ST_program/CellType_GP/DATA/spot_data_full.npz`

